In [1]:
import pandas as pd
import numpy as np

In [22]:
df_plays = pd.read_csv("data_sets/plays.csv")
df_pff = pd.read_csv("data_sets/PFFScoutingData.csv")
df_games = pd.read_csv("data_sets/games.csv")

In [20]:
df_track_2020 = pd.read_csv("data_sets/tracking2020.csv")
df_track_2019 = pd.read_csv("data_sets/tracking2019.csv")
df_track_2018 = pd.read_csv("data_sets/tracking2018.csv")

In [21]:
df_tracking = pd.concat([df_track_2018, df_track_2019, df_track_2020])

In [12]:
def process_play_data(df_plays):
    '''
    Get the necessary columns from the plays dataframe and select only the rows that are for kickoffs where there
    was actually a return on the play
    '''  
    kick_plays_df = df_plays[((df_plays["specialTeamsPlayType"] == "Kickoff") | (df_plays["specialTeamsPlayType"] == "Punt")) & (df_plays["specialTeamsResult"] == "Return")]
    kick_plays_df = kick_plays_df[["gameId", "playId", "specialTeamsPlayType","possessionTeam", "returnerId", "kickReturnYardage"]]  
    # Get rid of onside kicks
    kick_plays_df.dropna(axis=0, how='any', subset=['returnerId'], inplace=True)  
    # Get rid of kicks with multiple returners
    kick_plays_df.drop(kick_plays_df[kick_plays_df['returnerId'].str.contains(';')].index, inplace = True)  
    kick_plays_df["returnerId"] = kick_plays_df["returnerId"].astype('int')
    # Get rid of kicks with no return yards listed (likely because of fumble)
    kick_plays_df.dropna(axis=0, how='any', subset=['kickReturnYardage'], inplace=True) 
    
    return kick_plays_df

In [18]:
def process_pff_data(df_pff):
    pff_tackle_df = df_pff[["gameId", "playId", "missedTackler", "tackler"]]
    return pff_tackle_df
    

In [13]:
kick_plays_df = process_play_data(df_plays)

In [19]:
pff_tackle_df = process_pff_data(df_pff)

In [23]:
#using df_tracking to merge to jersey numbers

#selecting variables of interest & dropping duplicates - jersey # is constant throughout game
df_jerseyMap = df_tracking.drop_duplicates(subset = ["gameId", "team", "jerseyNumber", "nflId"])

#joining to games
df_jerseyMap = pd.merge(df_jerseyMap, df_games, left_on=['gameId'], right_on =['gameId'])

#getting name of team
conditions = [
    (df_jerseyMap['team'] == "home"),
    (df_jerseyMap['team'] != "home"),
]

values = [df_jerseyMap['homeTeamAbbr'], df_jerseyMap['visitorTeamAbbr']]

#adjusting jersey number so that it includes 0 when < 10
df_jerseyMap['team'] = np.select(conditions, values)

df_jerseyMap['jerseyNumber'] = df_jerseyMap['jerseyNumber'].astype(str)

df_jerseyMap.loc[df_jerseyMap['jerseyNumber'].map(len) < 4, 'jerseyNumber'] = "0"+df_jerseyMap.loc[df_jerseyMap['jerseyNumber'].map(len) < 4, 'jerseyNumber'].str[:2]

df_jerseyMap['jerseyNumber'] = df_jerseyMap['jerseyNumber'].str[:2]

#getting team and jersey
df_jerseyMap['teamJersey'] = df_jerseyMap['team'] + ' ' + df_jerseyMap['jerseyNumber'].str[:2]

#map to merge nflId to teamJersey
df_jerseyMap = df_jerseyMap[['gameId', 'nflId', 'teamJersey']]

df_jerseyMap = df_jerseyMap.sort_values(['gameId', 'nflId', 'teamJersey'])

df_jerseyMap.head(30)

,gameId,nflId,teamJersey
16378,2018090600,27091.0,ATL 03
16402,2018090600,29957.0,PHI 43
16379,2018090600,30826.0,PHI 94
16348,2018090600,32361.0,PHI 24
16380,2018090600,33083.0,PHI 56


In [24]:
pff_tackle_df.head()

,gameId,playId,missedTackler,tackler
0,2018090600,37,NaN,NaN
1,2018090600,366,PHI 57,PHI 54
2,2018090600,658,NaN,NaN
3,2018090600,677,ATL 83,ATL 27
4,2018090600,872,NaN,NaN


In [25]:
kick_plays_df.head()

,gameId,playId,specialTeamsPlayType,possessionTeam,returnerId,kickReturnYardage
1,2018090600,366,Punt,PHI,42450,5.0
3,2018090600,677,Kickoff,ATL,44979,30.0
9,2018090600,1606,Kickoff,ATL,44979,13.0
11,2018090600,1989,Punt,ATL,29957,8.0
14,2018090600,2599,Punt,ATL,29957,9.0


In [26]:
df_jerseyMap.head(30)

,gameId,nflId,teamJersey
16378,2018090600,27091.0,ATL 03
16402,2018090600,29957.0,PHI 43
16379,2018090600,30826.0,PHI 94
16348,2018090600,32361.0,PHI 24
16380,2018090600,33083.0,PHI 56
16349,2018090600,34465.0,PHI 27
16381,2018090600,34502.0,ATL 67
16382,2018090600,34836.0,PHI 77
16383,2018090600,35452.0,PHI 55
16350,2018090600,35722.0,ATL 82


In [27]:
merged_plays_pff = pd.merge(pff_tackle_df, kick_plays_df, how='inner')

In [49]:
merged_plays_pff.fillna(value="None", inplace=True)

In [79]:
merged_plays_pff[merged_plays_pff["gameId"] == 2019111711] 

,gameId,playId,missedTackler,tackler,specialTeamsPlayType,possessionTeam,returnerId,kickReturnYardage
2757,2019111711,54,None,LA 33,Kickoff,LA,39975,36.0
2758,2019111711,1663,None,LA 33,Kickoff,LA,44009,16.0
2759,2019111711,1993,None,CHI 55,Kickoff,CHI,45150,27.0
2760,2019111711,2587,None,CHI 84,Punt,CHI,45150,0.0
2761,2019111711,2829,None,CHI 27,Punt,CHI,45150,0.0
2762,2019111711,3643,LA 38; LA 51,LA 26,Kickoff,LA,39975,33.0


In [80]:
df_jerseyMap[(df_jerseyMap["gameId"] == 2019111711) & (df_jerseyMap["teamJersey"] == "LA 38")]

,gameId,nflId,teamJersey


In [ ]:
returner_id_list = []
tackler_id_list = []
tackle_results = []

i = 0
for index, row in merged_plays_pff.iterrows():
    print(index)
    print(row["gameId"], row["playId"])
    returner = row["returnerId"]
    missed_tacklers = row["missedTackler"]
    successful_tackler = row["tackler"]
    game = row["gameId"]
    play = row["playId"]
    if missed_tacklers != "None":
        missed_tacklers_list = missed_tacklers.split(";")
        for tackler in missed_tacklers_list:
            tackler = tackler.strip(" ")
            print(tackler)
            try:
                tackler_nfl_id = df_jerseyMap[(df_jerseyMap["gameId"] == game) & (df_jerseyMap["teamJersey"] == tackler)]["nflId"].values[0]
            except:
                continue
            returner_id_list.append(returner)
            tackler_id_list.append(int(tackler_nfl_id))
            tackle_results.append(0)
    if successful_tackler != "None":
        try:
            tackler_nfl_id = df_jerseyMap[(df_jerseyMap["gameId"] == game) & (df_jerseyMap["teamJersey"] == successful_tackler)]["nflId"].values[0]
        except:
            continue
        tackler_id_list.append(int(tackler_nfl_id))
        returner_id_list.append(returner)
        tackle_results.append(1)
    
    print("\n")
        
matchup_dict = {"returner_id": returner_id_list, "tackler_id": tackler_id_list, "results": tackle_results}
matchup_df = pd.DataFrame.from_dict(matchup_dict)

In [83]:
len(matchup_df)

6171